<a href="https://colab.research.google.com/github/kartik-5479/Gym_management_system/blob/main/GYM_MANAGEMENT_SYSTEM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paste & run this in Google Colab. You'll be prompted to paste your ngrok authtoken (hidden).

# Install required packages
!pip install -q flask pyngrok

import os, csv, hashlib, hmac, re, getpass, threading, time
from datetime import datetime, timezone, timedelta
from typing import Optional
from io import BytesIO

# Securely read authtoken from user (hidden input)
token = getpass.getpass("Paste your ngrok authtoken (it will be hidden): ").strip()
if not token:
    raise SystemExit("No authtoken provided — run again and paste your ngrok authtoken.")

# Configure pyngrok
from pyngrok import ngrok
ngrok.set_auth_token(token)
print("ngrok authtoken set. Starting Flask + ngrok...")

# --------------------------- Backend: your project code ---------------------------
DATA_DIR = '/content/gym_data'  # change to '/content/drive/MyDrive/gym_data' to persist to Drive
MEMBERS_FILE = os.path.join(DATA_DIR, 'members.csv')
ATTENDANCE_FILE = os.path.join(DATA_DIR, 'attendance.csv')

HASH_NAME = 'sha256'
ITERATIONS = 100_000
SALT_PREFIX = b'GYM-SALT-'
IST = timezone(timedelta(hours=5, minutes=30))


def ensure_data_dir():
    os.makedirs(DATA_DIR, exist_ok=True)


def _clean_phone(phone: str) -> str:
    return re.sub(r"\D", "", phone)


def readable_member_id(name: str, phone: str, dob: str) -> str:
    parts = [p for p in name.strip().split() if p]
    initials = 'X' if not parts else ''.join([p[0] for p in parts[:3]]).upper()
    digits = _clean_phone(phone)
    last4 = digits[-4:] if len(digits) >= 4 else digits.zfill(4)
    try:
        dt = datetime.strptime(dob.strip(), '%Y-%m-%d')
        dob_part = dt.strftime('%y%m%d')
    except Exception:
        digits_dob = re.sub(r"\D", "", dob)
        dob_part = digits_dob[-6:].zfill(6)
    return f"{initials}-{last4}-{dob_part}"


def hash_password(password: str, salt_material: str) -> str:
    salt = SALT_PREFIX + salt_material.encode('utf-8')
    dk = hashlib.pbkdf2_hmac(HASH_NAME, password.encode('utf-8'), salt, ITERATIONS)
    return dk.hex()


def verify_password(stored_hash: str, password_attempt: str, salt_material: str) -> bool:
    attempt_hash = hash_password(password_attempt, salt_material)
    return hmac.compare_digest(stored_hash, attempt_hash)


def init_files():
    ensure_data_dir()
    if not os.path.exists(MEMBERS_FILE):
        with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['id', 'name', 'phone', 'dob', 'email', 'password_hash', 'joined_on'])
    if not os.path.exists(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['timestamp', 'member_id', 'name', 'entry_type', 'notes'])


def read_members() -> dict:
    d = {}
    if not os.path.exists(MEMBERS_FILE):
        return d
    with open(MEMBERS_FILE, 'r', encoding='utf-8') as f:
        r = csv.DictReader(f)
        for row in r:
            d[row['id']] = row
    return d


def write_members(members: dict):
    with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['id', 'name', 'phone', 'dob', 'email', 'password_hash', 'joined_on'])
        for mid, info in members.items():
            w.writerow([mid, info.get('name',''), info.get('phone',''), info.get('dob',''), info.get('email',''), info.get('password_hash',''), info.get('joined_on','')])


def add_member(name: str, phone: str, dob: str, email: str, password: str) -> str:
    member_id = readable_member_id(name, phone, dob)
    joined_on = datetime.now(IST).isoformat()
    password_hash = hash_password(password, member_id)

    members = read_members()
    if member_id in members:
        existing = members[member_id]
        if existing['name'].strip().lower() == name.strip().lower() and _clean_phone(existing['phone']) == _clean_phone(phone):
            members[member_id].update({'name': name, 'phone': phone, 'dob': dob, 'email': email})
            write_members(members)
            return member_id
        import hashlib as _hl
        suffix = _hl.sha256((name+phone+dob).encode('utf-8')).hexdigest()[-3:].upper()
        member_id = f"{member_id}-{suffix}"

    with open(MEMBERS_FILE, 'a', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow([member_id, name, phone, dob, email, password_hash, joined_on])
    return member_id


def log_attendance(member_id: str, name: str, entry_type: str='IN', notes: str=''):
    ts = datetime.now(IST).isoformat()
    with open(ATTENDANCE_FILE, 'a', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow([ts, member_id, name, entry_type, notes])


def read_attendance() -> list:
    if not os.path.exists(ATTENDANCE_FILE):
        return []
    with open(ATTENDANCE_FILE, 'r', encoding='utf-8') as f:
        return list(csv.DictReader(f))


init_files()

# --------------------------- Flask app (uses ngrok) ---------------------------
from flask import Flask, request, redirect, url_for, render_template_string, flash, get_flashed_messages, send_file, abort

app = Flask(__name__)
app.secret_key = os.environ.get('FLASK_SECRET', 'dev-key-for-demo')  # change for production

# Using Bootstrap 5 for a clean, professional frontend
BASE_HTML = """
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Gym Attendance Portal</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-PLACEHOLDER" crossorigin="anonymous">
    <style>
      body { padding-top: 70px; }
      .brand { font-weight:700; letter-spacing:0.6px }
      .small-muted { font-size:0.9rem; color: #6c757d }
      .table-fit td, .table-fit th { white-space: nowrap }
    </style>
  </head>
  <body>

    <nav class="navbar navbar-expand-lg navbar-dark bg-primary fixed-top">
      <div class="container-fluid">
        <a class="navbar-brand brand" href="/">GymPortal</a>
        <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navcollapse">
          <span class="navbar-toggler-icon"></span>
        </button>
        <div class="collapse navbar-collapse" id="navcollapse">
          <ul class="navbar-nav me-auto mb-2 mb-lg-0">
            <li class="nav-item"><a class="nav-link" href="/register">Register</a></li>
            <li class="nav-item"><a class="nav-link" href="/login">Entry</a></li>
            <li class="nav-item"><a class="nav-link" href="/exit">Exit</a></li>
            <li class="nav-item"><a class="nav-link" href="/members">Members</a></li>
            <li class="nav-item"><a class="nav-link" href="/attendance">Attendance</a></li>
          </ul>
          <span class="navbar-text small-muted">Timestamps in IST (UTC+5:30)</span>
        </div>
      </div>
    </nav>

    <main class="container">
      {% with messages = get_flashed_messages() %}
        {% if messages %}
          <div class="mt-2">
            {% for m in messages %}
              <div class="alert alert-info alert-dismissible fade show" role="alert">
                {{m}}
                <button type="button" class="btn-close" data-bs-dismiss="alert" aria-label="Close"></button>
              </div>
            {% endfor %}
          </div>
        {% endif %}
      {% endwith %}

      {{ body | safe }}

    </main>

    <footer class="text-center mt-4 mb-4 small-muted">
      <div class="container">&copy; {{year}} GymPortal — built with Flask</div>
    </footer>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/js/bootstrap.bundle.min.js" integrity="sha384-PLACEHOLDER" crossorigin="anonymous"></script>
    <script>
      // small client-side table filter
      function filterTable(inputId, tableId) {
        const input = document.getElementById(inputId);
        const filter = input.value.toLowerCase();
        const rows = document.querySelectorAll(`#${tableId} tbody tr`);
        rows.forEach(r => {
          r.style.display = r.innerText.toLowerCase().includes(filter) ? '' : 'none';
        });
      }
    </script>
  </body>
</html>
"""

HOME_BODY = """
<div class="row">
  <div class="col-md-8">
    <div class="card mb-3">
      <div class="card-body">
        <h3 class="card-title">Welcome to GymPortal</h3>
        <p class="card-text">A clean, secure attendance system for small gyms and studios. Use the menu to register members and log entries/exits.</p>
        <div class="d-flex gap-2">
          <a href="/register" class="btn btn-primary">Register Member</a>
          <a href="/login" class="btn btn-outline-primary">Member Entry</a>
          <a href="/exit" class="btn btn-outline-secondary">Member Exit</a>
        </div>
      </div>
    </div>

    <div class="card">
      <div class="card-body">
        <h5>Quick stats</h5>
        <ul>
          <li>Members stored: <strong>{{member_count}}</strong></li>
          <li>Attendance records: <strong>{{attendance_count}}</strong></li>
        </ul>
      </div>
    </div>
  </div>
  <div class="col-md-4">
    <div class="card text-center">
      <div class="card-body">
        <h5 class="card-title">Today's last entry</h5>
        {% if last_entry %}
          <p class="mb-0"><strong>{{last_entry.name}}</strong></p>
          <p class="small-muted mb-0">{{last_entry.timestamp}}</p>
        {% else %}
          <p class="mb-0 small-muted">No entries yet</p>
        {% endif %}
      </div>
    </div>
  </div>
</div>
"""

REGISTER_BODY = """
<div class="card">
  <div class="card-body">
    <h3 class="card-title">Register New Member</h3>
    <form method="post" class="row g-3">
      <div class="col-md-6">
        <label class="form-label">Full name</label>
        <input name="name" class="form-control" required>
      </div>
      <div class="col-md-6">
        <label class="form-label">Phone (digits)</label>
        <input name="phone" class="form-control" required>
      </div>
      <div class="col-md-4">
        <label class="form-label">DOB (YYYY-MM-DD)</label>
        <input name="dob" class="form-control" placeholder="1998-05-20" required>
      </div>
      <div class="col-md-8">
        <label class="form-label">Email (optional)</label>
        <input name="email" class="form-control" type="email">
      </div>
      <div class="col-md-6">
        <label class="form-label">Password</label>
        <input name="password" class="form-control" type="password" required>
      </div>
      <div class="col-12 mt-2">
        <button type="submit" class="btn btn-success">Register</button>
        <a href="/" class="btn btn-link">Cancel</a>
      </div>
    </form>
  </div>
</div>
"""

LOGIN_BODY = """
<div class="card">
  <div class="card-body">
    <h3 class="card-title">Member Entry (Login)</h3>
    <form method="post" class="row g-3">
      <div class="col-md-6">
        <label class="form-label">Member ID</label>
        <input name="member_id" class="form-control" required>
      </div>
      <div class="col-md-6">
        <label class="form-label">Password</label>
        <input name="password" class="form-control" type="password" required>
      </div>
      <div class="col-12 mt-2">
        <button type="submit" class="btn btn-primary">Enter (Log IN)</button>
        <a href="/" class="btn btn-link">Cancel</a>
      </div>
    </form>
  </div>
</div>
"""

EXIT_BODY = """
<div class="card">
  <div class="card-body">
    <h3 class="card-title">Member Exit</h3>
    <form method="post" class="row g-3">
      <div class="col-md-6">
        <label class="form-label">Member ID</label>
        <input name="member_id" class="form-control" required>
      </div>
      <div class="col-md-6">
        <label class="form-label">Password</label>
        <input name="password" class="form-control" type="password" required>
      </div>
      <div class="col-12 mt-2">
        <button type="submit" class="btn btn-warning">Exit (Log OUT)</button>
        <a href="/" class="btn btn-link">Cancel</a>
      </div>
    </form>
  </div>
</div>
"""

TABLE_BODY = """
<div class="card">
  <div class="card-body">
    <h3 class="card-title">{{title}}</h3>
    <div class="mb-3 d-flex justify-content-between align-items-center">
      <input id="filterInput" oninput="filterTable('filterInput','dataTable')" class="form-control form-control-sm w-50" placeholder="Type to filter table">
      <div>
        {% if export_members %}
          <a href="/export/members" class="btn btn-sm btn-outline-success">Export Members CSV</a>
        {% endif %}
        {% if export_attendance %}
          <a href="/export/attendance" class="btn btn-sm btn-outline-success">Export Attendance CSV</a>
        {% endif %}
        <a href="/" class="btn btn-sm btn-outline-secondary">Back</a>
      </div>
    </div>
    <div class="table-responsive">
      <table id="dataTable" class="table table-striped table-hover table-fit">
        <thead class="table-light"><tr>{% for h in headers %}<th>{{h}}</th>{% endfor %}</tr></thead>
        <tbody>
        {% for row in rows %}
          <tr>{% for h in headers %}<td>{{row[h]}}</td>{% endfor %}</tr>
        {% endfor %}
        </tbody>
      </table>
    </div>
  </div>
</div>
"""

from flask import render_template_string

@app.route('/')
def home():
    members = read_members()
    attendance = read_attendance()
    last_entry = attendance[-1] if attendance else None
    body = render_template_string(HOME_BODY, member_count=len(members), attendance_count=len(attendance), last_entry=last_entry)
    return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)


@app.route('/register', methods=['GET','POST'])
def register():
    if request.method == 'GET':
        body = REGISTER_BODY
        return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)
    name = request.form.get('name','').strip()
    phone = request.form.get('phone','').strip()
    dob = request.form.get('dob','').strip()
    email = request.form.get('email','').strip()
    password = request.form.get('password','').strip()
    if not (name and phone and dob and password):
        flash("Please fill required fields")
        return redirect(url_for('register'))
    mid = add_member(name, phone, dob, email, password)
    flash(f"Registered {name} with ID: {mid}")
    return redirect(url_for('register'))


@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'GET':
        body = LOGIN_BODY
        return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('login'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'IN', notes='Web-entry')
        flash(f"Entry logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('login'))


@app.route('/exit', methods=['GET','POST'])
def exit_member():
    if request.method == 'GET':
        body = EXIT_BODY
        return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('exit_member'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'OUT', notes='Web-exit')
        flash(f"Exit logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED_EXIT', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('exit_member'))


@app.route('/members')
def members_page():
    members = list(read_members().values())
    headers = ['id','name','phone','dob','email','joined_on']
    body = render_template_string(TABLE_BODY, title="Registered Members", headers=headers, rows=members, export_members=True, export_attendance=False)
    return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)


@app.route('/attendance')
def attendance_page():
    rows = read_attendance()
    headers = ['timestamp','member_id','name','entry_type','notes']
    rows = list(reversed(rows))
    body = render_template_string(TABLE_BODY, title="Attendance Log (latest first)", headers=headers, rows=rows, export_members=False, export_attendance=True)
    return render_template_string(BASE_HTML, body=body, year=datetime.now().year, get_flashed_messages=get_flashed_messages)


# CSV export endpoints
@app.route('/export/members')
def export_members_csv():
    if not os.path.exists(MEMBERS_FILE):
        abort(404, "Members file not found")
    # Stream file contents directly back to client
    return send_file(MEMBERS_FILE, as_attachment=True, download_name='members.csv', mimetype='text/csv')

@app.route('/export/attendance')
def export_attendance_csv():
    if not os.path.exists(ATTENDANCE_FILE):
        abort(404, "Attendance file not found")
    return send_file(ATTENDANCE_FILE, as_attachment=True, download_name='attendance.csv', mimetype='text/csv')


def start_tunnel_and_app():
    public_url = ngrok.connect(5000).public_url
    print("Public URL:", public_url)
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=start_tunnel_and_app, daemon=True)
thread.start()
# give ngrok a second to connect
time.sleep(3)
print("If you see 'Public URL:' above, open it to use the web UI.")





Paste your ngrok authtoken (it will be hidden): ··········
ngrok authtoken set. Starting Flask + ngrok...
Public URL: https://inscriptive-placably-katie.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


If you see 'Public URL:' above, open it to use the web UI.
